In [77]:
import pandas as pd
import glob

# Define the column names for the first two rows
column_names = ['Year', 'January', 'January_rank', 'February', 'February_rank', 'March', 'March_rank',
                'April', 'April_rank', 'May', 'May_rank', 'June', 'June_rank', 'July', 'July_rank',
                'August', 'August_rank', 'September', 'September_rank', 'October', 'October_rank',
                'November', 'November_rank', 'December', 'December_rank']

# Define the month order for sorting
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

# Get a list of all area and extent CSV files
area_files = glob.glob('data/N_Sea_Ice_Index_Regional_Monthly_Data_G02135_v3.0*Area*.csv')
extent_files = glob.glob('data/N_Sea_Ice_Index_Regional_Monthly_Data_G02135_v3.0*Extent*.csv')

# Function to load and process files
def load_and_process_files(files, feature):
    dataframes = []
    for file in files:
        df = pd.read_csv(file, names=column_names, header=2)  # Skip the first two header rows
        region_name = file.split(' - ')[1].split('-')[0]  # Extract region name from file name
        # Rename columns to include region name and feature, and drop rank columns
        df = df.rename(columns={col: f"{region_name}_{feature}_{col}" for col in df.columns if col != 'Year' and 'rank' not in col})
        rank_columns = [col for col in df.columns if 'rank' in col]
        df = df.drop(columns=rank_columns, errors='ignore')
        dataframes.append(df.set_index('Year'))
    return pd.concat(dataframes, axis=1)

# Load and process area and extent files
area_df = load_and_process_files(area_files, 'area')
extent_df = load_and_process_files(extent_files, 'extent')

# Merge area and extent dataframes on 'Year'
merged_df = pd.merge(area_df, extent_df, on='Year', how='outer')

# Reset the index to make the 'Year' column a regular column again
merged_df.reset_index(inplace=True)

# Sort the columns by region, feature, and month, keeping 'Year' at the beginning
def sort_key(col):
    if col == 'Year':
        return (0, '')
    parts = col.split('_')
    region = parts[0]
    feature = parts[1]
    month = parts[2]
    if month not in month_order:
        return (1, region, 0, feature)  # Handle unexpected month names
    return (1, region, month_order.index(month), feature)

sorted_columns = ['Year'] + sorted([col for col in merged_df.columns if col != 'Year'], key=sort_key)
merged_df = merged_df[sorted_columns]

# Save the merged dataframe to a new CSV file
merged_df.to_csv('merged_sea_ice_data.csv', index=False)

print("Merged data saved to 'merged_sea_ice_data.csv'")

Merged data saved to 'merged_sea_ice_data.csv'


In [78]:
merged_df.head()

,Year,Baffin_area_January,Baffin_extent_January,Baffin_area_February,Baffin_extent_February,Baffin_area_March,Baffin_extent_March,Baffin_area_April,Baffin_extent_April,Baffin_area_May,...,St_area_August,St_extent_August,St_area_September,St_extent_September,St_area_October,St_extent_October,St_area_November,St_extent_November,St_area_December,St_extent_December
0,1978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2754.204,4997.086,14018.369,30889.921
1,1979,840498.561,1090625.815,922075.196,1157634.866,1031144.947,1299583.477,904254.808,1137194.680,789391.835,...,0.0,0.0,0.0,0.0,0.0,0.0,2554.961,4163.668,15441.414,35715.843
2,1980,870924.628,1104871.464,958526.404,1215999.908,1036412.094,1284177.396,983828.079,1221357.854,813411.383,...,0.0,0.0,0.0,0.0,0.0,0.0,2342.721,3962.027,20221.155,42251.682
3,1981,928643.884,1122447.922,1023671.618,1282707.945,951153.228,1117472.461,966782.270,1147369.831,807937.058,...,0.0,0.0,0.0,0.0,0.0,0.0,2436.574,4085.028,11977.659,27758.715
4,1982,853589.217,1080833.934,1132306.390,1403639.746,1211558.994,1487905.095,1099459.619,1345921.021,961648.008,...,0.0,0.0,0.0,0.0,0.0,0.0,3018.179,4793.096,14073.623,31893.900


In [90]:
# Ensure all columns (except 'Year') are numeric
for col in merged_df.columns:
    if col != 'Year':
        merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce')

# Fill missing values with the mean of each column
merged_df_filled_mean = merged_df.fillna(merged_df.mean())

# Check for any remaining missing values
missing_values_after_fill = merged_df_filled_mean.isnull().sum()
print("Missing values in each column after filling with the mean:")
print(missing_values_after_fill)

# Print the DataFrame after filling missing values with the mean
print("DataFrame after filling missing values with the mean:")
print(merged_df_filled_mean.head())

Missing values in each column after filling with the mean:
Year                      0
Baffin_area_January       0
Baffin_extent_January     0
Baffin_area_February      0
Baffin_extent_February    0
                         ..
St_extent_October         0
St_area_November          0
St_extent_November        0
St_area_December          0
St_extent_December        0
Length: 337, dtype: int64
DataFrame after filling missing values with the mean:
   Year  Baffin_area_January  Baffin_extent_January  Baffin_area_February  \
0  1978        921282.375711           1.125875e+06          1.080705e+06   
1  1979        840498.561000           1.090626e+06          9.220752e+05   
2  1980        870924.628000           1.104871e+06          9.585264e+05   
3  1981        928643.884000           1.122448e+06          1.023672e+06   
4  1982        853589.217000           1.080834e+06          1.132306e+06   

   Baffin_extent_February  Baffin_area_March  Baffin_extent_March  \
0            1.310693